In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import boto3
import json
import itertools
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
cs_clnt = boto3.client("cloudsearch")

Found credentials in shared credentials file: ~/.aws/credentials


<IPython.core.display.Javascript object>

In [4]:
domain_name = "mpn-mfg-2"

<IPython.core.display.Javascript object>

In [5]:
response = cs_clnt.describe_domains(DomainNames=[domain_name])
doc_ep = response["DomainStatusList"][0]["DocService"]["Endpoint"]
search_ep = response["DomainStatusList"][0]["SearchService"]["Endpoint"]
print("doc_ep: ", doc_ep)
print("search_ep: ", search_ep)

doc_ep:  doc-mpn-mfg-2-35jf2boyu4jash2rk5vrcgpa34.us-west-2.cloudsearch.amazonaws.com
search_ep:  search-mpn-mfg-2-35jf2boyu4jash2rk5vrcgpa34.us-west-2.cloudsearch.amazonaws.com


<IPython.core.display.Javascript object>

In [6]:
# doc_client = boto3.client("cloudsearchdomain", endpoint_url=f"https://{doc_ep}")

<IPython.core.display.Javascript object>

In [7]:
search_ep = (
    "search-mpn-mfg-2-35jf2boyu4jash2rk5vrcgpa34.us-west-2.cloudsearch.amazonaws.com"
)

<IPython.core.display.Javascript object>

In [8]:
src_clnt = boto3.client("cloudsearchdomain", endpoint_url=f"https://{search_ep}")

<IPython.core.display.Javascript object>

In [9]:
def parse_response(response):
    # nhits = response["hits"]["found"]
    nhits = len(response["hits"]["hit"])
    # print(nhits)
    # df.explode("sub_commodity_ihs")
    df = pd.DataFrame()
    for i in range(nhits):
        # print(response["hits"]["hit"][i])
        row = response["hits"]["hit"][i]["fields"]
        df = df.append(row, ignore_index=True)

    # df = df.fillna("").apply(list)
    df = df.apply(lambda s: s.fillna({i: ["Unassigned"] for i in df.index}))

    for col in df.columns:
        df[col] = list(itertools.chain(*df[col]))

    cols = [
        "mpn",
        "mpn_alphanum",
        "manufacturer_ihs",
        "manufacturer_ld",
        "commodity_ihs",
        "commodity_ld",
        "sub_commodity_ihs",
        "sub_commodity_ld",
        "manufacturer_support_ihs",
        "manufacturer_evidence_ihs",
        "commodity_support_ihs",
        "commodity_evidence_ihs",
        "sub_commodity_support_ihs",
        "sub_commodity_evidence_ihs",
        "description_ihs",
        "date_ihs",
    ]

    df = df[cols]
    return df

<IPython.core.display.Javascript object>

#### single mpn

In [10]:
%%time
query = "NRC04J164TR50F"
query = 'RMCF1/16S3.92K1%'

response = src_clnt.search(
    query=query,
    #queryParser="simple",
    partial=True,
    queryOptions='{"fields":["_id"]}',
)
parse_response(response)

CPU times: user 80.2 ms, sys: 5.93 ms, total: 86.1 ms
Wall time: 924 ms


,mpn,mpn_alphanum,manufacturer_ihs,manufacturer_ld,commodity_ihs,commodity_ld,sub_commodity_ihs,sub_commodity_ld,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,description_ihs,date_ihs
0,RMCF1/16S3.92K1%,RMCF116S392K1,SEI ELEK,STACKPOLE ELECTRONICS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,8,100,8,100,ihs description,2022-05-01T00:00:00Z


<IPython.core.display.Javascript object>

#### multiple mpns

In [11]:
%%time

mpns = ["NRC04J164TR50F", "CRCW120633K0JNECIF", "RCA040218R0FKEE", "ABCD"]

query = ""
for mpn in mpns:
    query = mpn + " | " + query
print(query)

response = src_clnt.search(
    query=query,
    #queryParser="simple",
    #partial=True,
    queryOptions='{"fields":["_id"]}',
)
parse_response(response)

ABCD | RCA040218R0FKEE | CRCW120633K0JNECIF | NRC04J164TR50F | 
CPU times: user 56.4 ms, sys: 2.33 ms, total: 58.7 ms
Wall time: 316 ms


,mpn,mpn_alphanum,manufacturer_ihs,manufacturer_ld,commodity_ihs,commodity_ld,sub_commodity_ihs,sub_commodity_ld,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,description_ihs,date_ihs
0,CRCW120633K0JNECIF,CRCW120633K0JNECIF,VISHAY,VISHAY,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z
1,NRC04J164TR50F,NRC04J164TR50F,NIC,NIC COMPONENTS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z
2,RCA040218R0FKEE,RCA040218R0FKEE,VISHAY,VISHAY,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z


<IPython.core.display.Javascript object>

#### prefix

In [12]:
%%time


query = "NRC04J5*"

response = src_clnt.search(
    query=query,
    #queryParser="simple",
    partial=True,
    queryOptions='{"fields":["_id"]}',
)
parse_response(response)

CPU times: user 50.3 ms, sys: 1.62 ms, total: 52 ms
Wall time: 260 ms


,mpn,mpn_alphanum,manufacturer_ihs,manufacturer_ld,commodity_ihs,commodity_ld,sub_commodity_ihs,sub_commodity_ld,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,description_ihs,date_ihs
0,NRC04J563TRF,NRC04J563TRF,NIC,NIC COMPONENTS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,3,66.0,5,100,5,100,ihs description,2022-05-01T00:00:00Z
1,NRC04J5R6TR10F,NRC04J5R6TR10F,NIC,NIC COMPONENTS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z


<IPython.core.display.Javascript object>

#### prefix & postfix

In [13]:
%%time
query = "NRC04J*TR50F"

response = src_clnt.search(
    query=query,
    queryParser="lucene",
    partial=True,
    queryOptions='{"fields":["_id"]}',
)
parse_response(response)

CPU times: user 45.9 ms, sys: 1.78 ms, total: 47.7 ms
Wall time: 360 ms


,mpn,mpn_alphanum,manufacturer_ihs,manufacturer_ld,commodity_ihs,commodity_ld,sub_commodity_ihs,sub_commodity_ld,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,description_ihs,date_ihs
0,NRC04J164TR50F,NRC04J164TR50F,NIC,NIC COMPONENTS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z
1,NRC04J752HTR50F,NRC04J752HTR50F,NIC,NIC COMPONENTS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z


<IPython.core.display.Javascript object>

#### prefix & postfix with fixed number of chars in the middle

In [14]:
%%time
query = "NRC04J???TR50F"

response = src_clnt.search(
    query=query,
    queryParser="lucene",
    partial=True,
    queryOptions='{"fields":["_id"]}',
)
parse_response(response)

CPU times: user 44.8 ms, sys: 1.91 ms, total: 46.7 ms
Wall time: 259 ms


,mpn,mpn_alphanum,manufacturer_ihs,manufacturer_ld,commodity_ihs,commodity_ld,sub_commodity_ihs,sub_commodity_ld,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,description_ihs,date_ihs
0,NRC04J164TR50F,NRC04J164TR50F,NIC,NIC COMPONENTS,RESISTORS,EE_PASSIVES,FIXED RESISTORS,FIXED RESISTORS,2,100.0,2,100,2,100,ihs description,2022-05-01T00:00:00Z


<IPython.core.display.Javascript object>

#### low evidence

In [15]:
%%time

query = "(range field = manufacturer_evidence_ihs {,50])"
                   
response = src_clnt.search(
    query=query,
    queryParser="structured",
    #partial=True,
    #queryOptions='{"fields":["_id"]}',
    #queryOptions='{"fields":["manufacturer_evidence_ihs"]}',
)
parse_response(response)

CPU times: user 73.9 ms, sys: 2.04 ms, total: 75.9 ms
Wall time: 297 ms


,mpn,mpn_alphanum,manufacturer_ihs,manufacturer_ld,commodity_ihs,commodity_ld,sub_commodity_ihs,sub_commodity_ld,manufacturer_support_ihs,manufacturer_evidence_ihs,commodity_support_ihs,commodity_evidence_ihs,sub_commodity_support_ihs,sub_commodity_evidence_ihs,description_ihs,date_ihs
0,2N7002KDW,2N7002KDW,DIODES,DIODES,TRANSISTORS,DISCRETE_SEMICONDUCTOR_DEVICES,SMALL SIGNAL FIELD-EFFECT TRANSISTORS,TRANSISTORS,8,50.0,8,100,8,100,ihs description,2022-05-01T00:00:00Z
1,742792117,742792117,WUERTH,WURTH,PASSIVE FILTERS,EE_PASSIVES,DATA LINE FILTERS,DATA LINE FILTERS,8,50.0,8,100,8,100,ihs description,2022-05-01T00:00:00Z
2,B0530WS,B0530WS,TAIWAN SEMI,TSMC,DIODES,DISCRETE_SEMICONDUCTOR_DEVICES,RECTIFIER DIODES,DIODES,8,50.0,12,100,12,100,ihs description,2022-05-01T00:00:00Z
3,B0530WS-*,B0530WS,TAIWAN SEMI,TSMC,DIODES,DISCRETE_SEMICONDUCTOR_DEVICES,RECTIFIER DIODES,DIODES,8,50.0,12,100,12,100,ihs description,2022-05-01T00:00:00Z
4,BAS16H,BAS16H,GALAXY SEMI,GALAXY SEMICONDUCTOR,DIODES,DISCRETE_SEMICONDUCTOR_DEVICES,RECTIFIER DIODES,DIODES,8,50.0,10,100,10,100,ihs description,2022-05-01T00:00:00Z
5,BAT54A-T,BAT54AT,RECTRON,RECTRON,DIODES,DISCRETE_SEMICONDUCTOR_DEVICES,RECTIFIER DIODES,DIODES,10,40.0,10,100,10,100,ihs description,2022-05-01T00:00:00Z
6,BAT54AT,BAT54AT,RECTRON,RECTRON,DIODES,DISCRETE_SEMICONDUCTOR_DEVICES,RECTIFIER DIODES,DIODES,10,40.0,10,100,10,100,ihs description,2022-05-01T00:00:00Z
7,BC817-40,BC81740,CENTRAL SEMI,CENTRAL SEMICONDUCTOR,TRANSISTORS,DISCRETE_SEMICONDUCTOR_DEVICES,SMALL SIGNAL BIPOLAR TRANSISTORS,TRANSISTORS,14,28.0,14,100,14,100,ihs description,2022-05-01T00:00:00Z
8,BC817.40,BC81740,CENTRAL SEMI,CENTRAL SEMICONDUCTOR,TRANSISTORS,DISCRETE_SEMICONDUCTOR_DEVICES,SMALL SIGNAL BIPOLAR TRANSISTORS,TRANSISTORS,14,28.0,14,100,14,100,ihs description,2022-05-01T00:00:00Z
9,BC81740,BC81740,CENTRAL SEMI,CENTRAL SEMICONDUCTOR,TRANSISTORS,DISCRETE_SEMICONDUCTOR_DEVICES,SMALL SIGNAL BIPOLAR TRANSISTORS,TRANSISTORS,14,28.0,14,100,14,100,ihs description,2022-05-01T00:00:00Z


<IPython.core.display.Javascript object>

CENTRAL SEMI    4
MCC             2
CDIL            2
ZETEX           2
PANJIT          2
NEXPERIA        2